# Data analysis of CIA World Factbook, a compendium of statistics about all of the countries on Earth. 
Demographic information in the data includes:
* population — the global population.
* population_growth — the annual population growth rate, as a percentage.
* area — the total land and water area.


In [3]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [4]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


[('table', 'sqlite_sequence', 'sqlite_sequence', 3, 'CREATE TABLE sqlite_sequence(name,seq)'),
 ('table', 'facts', 'facts', 47, 'CREATE TABLE "facts" ("id" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, "code" varchar(255) NOT NULL, "name" varchar(255) NOT NULL, "area" integer, "a ... (4 characters truncated) ... land" integer, "area_water" integer, "population" integer, "population_growth" float, "birth_rate" float, "death_rate" float, "migration_rate" float)')]

In [9]:
%%sql
SELECT *
FROM facts
LIMIT 5



Done.


[(1, 'af', 'Afghanistan', 652230, 652230, 0, 32564342, 2.32, 38.57, 13.89, 1.51),
 (2, 'al', 'Albania', 28748, 27398, 1350, 3029278, 0.3, 12.92, 6.58, 3.3),
 (3, 'ag', 'Algeria', 2381741, 2381741, 0, 39542166, 1.84, 23.67, 4.31, 0.92),
 (4, 'an', 'Andorra', 468, 468, 0, 85580, 0.12, 8.13, 6.96, 0.0),
 (5, 'ao', 'Angola', 1246700, 1246700, 0, 19625353, 2.78, 38.78, 11.49, 0.46)]

In [10]:
%%sql
SELECT MIN(population), MAX(population),
    MIN(population_growth), MAX(population_growth)
    FROM facts;

Done.


[(0, 7256490011, 0.0, 4.02)]

In [12]:
%%sql
SELECT name
FROM  facts
WHERE population =(SELECT MIN(population) FROM facts);

Done.


[('Antarctica',)]

In [13]:
%%sql
SELECT name
FROM  facts
WHERE population =(SELECT MAX(population) FROM facts);

Done.


[('World',)]

- Exclude 'Antarctica', 'World' from the summary statistics

In [16]:
%%sql
SELECT MIN(population), MAX(population),
    MIN(population_growth), MAX(population_growth)
    FROM facts
    WHERE name NOT IN ('Antarctica', 'World');

Done.


[(48, 1367485388, 0.0, 4.02)]

In [21]:
%%sql
SELECT ROUND(AVG(population)) AS avg_population, ROUND(AVG(area),2) AS avg_land_mass
FROM facts

Done.


[(62094928.0, 555093.55)]

In [23]:
%%sql
SELECT name, population, area
FROM facts
WHERE 
(population > (SELECT ROUND(AVG(population)) FROM facts)
AND area < (SELECT ROUND(AVG(area),2) FROM facts));

Done.


[('Bangladesh', 168957745, 148460),
 ('Germany', 80854408, 357022),
 ('Japan', 126919659, 377915),
 ('Philippines', 100998376, 300000),
 ('Thailand', 67976405, 513120),
 ('United Kingdom', 64088222, 243610),
 ('Vietnam', 94348835, 331210)]

- Which country has the most people? Which country has the highest growth rate?

In [37]:
%%sql
SELECT name, population
FROM facts
WHERE population = (
    SELECT MAX(population) FROM facts WHERE name != "World")

                   

Done.


[('China', 1367485388)]

In [43]:
%%sql
SELECT name, population_growth
FROM facts
WHERE population_growth = (SELECT MAX(population_growth) FROM facts);

Done.


[('South Sudan', 4.02)]

- Which countries have the highest ratios of water to land? Which countries have more water than land?

In [50]:
%%sql
SELECT name, CAST(area_water AS FLOAT)/CAST(area_land AS FLOAT) AS water_land_ratio
FROM facts
WHERE water_land_ratio = (SELECT MAX(CAST(area_water AS FLOAT)/CAST(area_land AS FLOAT))
FROM facts)

Done.


[('British Indian Ocean Territory', 905.6666666666666)]

- Which countries will add the most people to their populations next year?

In [60]:
%%sql
SELECT name, (population*population_growth)/100 AS pop_incr
FROM facts
WHERE pop_incr = (
SELECT MAX((population*population_growth)/100) AS max_pop_incr
FROM facts
WHERE name != 'World')

Done.


[('India', 15270686.1248)]

- Which countries have a higher death rate than birth rate?

In [61]:
%%sql
SELECT name, death_rate, birth_rate
FROM facts
WHERE (name != 'World' AND death_rate > birth_rate)

Done.


[('Austria', 9.42, 9.41),
 ('Belarus', 13.36, 10.7),
 ('Bosnia and Herzegovina', 9.75, 8.87),
 ('Bulgaria', 14.44, 8.92),
 ('Croatia', 12.18, 9.45),
 ('Czech Republic', 10.34, 9.63),
 ('Estonia', 12.4, 10.51),
 ('Germany', 11.42, 8.47),
 ('Greece', 11.09, 8.66),
 ('Hungary', 12.73, 9.16),
 ('Italy', 10.19, 8.74),
 ('Japan', 9.51, 7.93),
 ('Latvia', 14.31, 10.0),
 ('Lithuania', 14.27, 10.1),
 ('Moldova', 12.59, 12.0),
 ('Monaco', 9.24, 6.65),
 ('Poland', 10.19, 9.74),
 ('Portugal', 11.02, 9.27),
 ('Romania', 11.9, 9.14),
 ('Russia', 13.69, 11.6),
 ('Serbia', 13.66, 9.08),
 ('Slovenia', 11.37, 8.42),
 ('Ukraine', 14.46, 10.72),
 ('Saint Pierre and Miquelon', 9.72, 7.42)]

- Which countries have the highest population/area ratio, and how does it compare to list we found in the previous screen?

In [69]:
%%sql
SELECT name, MAX(ROUND(CAST(population AS FLOAT)/area_land,2)) AS pop_land_ratio
FROM facts
WHERE name != 'World'

Done.


[('Macau', 21168.96)]